In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Understand the structure of the dSprites dataset
data = np.load('datasets/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz',
                encoding='latin1', # These two
                allow_pickle=True) # are for loading metadata
data

NpzFile 'datasets/dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz' with keys: metadata, imgs, latents_classes, latents_values

In [7]:
latent_sizes = torch.Tensor(data["metadata"][()]["latents_sizes"])
# torch.index_select(latent_sizes, 0,torch.tensor([0,1,2,3]))

In [13]:
latent_sizes

tensor([ 1.,  3.,  6., 40., 32., 32.])

In [15]:
sample = torch.prod(latent_sizes.int())/torch.cumprod(latent_sizes.int(),axis=0)

In [14]:
samples = torch.rand(10, len(latent_sizes))
new_samples = (samples * (latent_sizes)).long().floor()

In [22]:
indices = torch.tensor(np.dot(new_samples.numpy(), sample.numpy())).int()
indices

tensor([604108, 267977, 316720, 374397, 475746, 341568, 229797, 370521, 565354,
        457936], dtype=torch.int32)

In [16]:
new_samples

tensor([[ 0,  2,  2, 29, 30, 12],
        [ 0,  1,  0, 21, 22,  9],
        [ 0,  1,  1, 29,  9, 16],
        [ 0,  1,  3,  5, 19, 29],
        [ 0,  1,  5, 24, 19,  2],
        [ 0,  1,  2, 13, 18,  0],
        [ 0,  0,  5, 24, 13,  5],
        [ 0,  1,  3,  1, 26, 25],
        [ 0,  2,  1, 32,  3, 10],
        [ 0,  1,  5,  7,  6, 16]])

In [59]:
new_samples[:,[1, 3 ,4]]

tensor([[ 1,  5,  2],
        [ 2, 21,  2],
        [ 1, 35, 16],
        [ 2,  5, 15],
        [ 0, 20, 26],
        [ 1, 26, 11],
        [ 2, 24, 23],
        [ 0, 37,  1],
        [ 1, 23, 26],
        [ 0, 35, 24]])

In [47]:
latent_sizes.shape

torch.Size([6])

In [35]:
# Get a list of random indices
indices = np.random.randint(0, 200, 5)
indices

array([138,  22,  84, 122, 104])

In [33]:
factor_sizes = np.array(data["metadata"][()]["latents_sizes"])
np.prod(factor_sizes)/ np.cumprod(factor_sizes) # number of images per factor

array([7.3728e+05, 2.4576e+05, 4.0960e+04, 1.0240e+03, 3.2000e+01,
       1.0000e+00])

In [32]:
imgs.shape

ValueError: can only specify one unknown dimension

In [25]:
data['metadata'][()]

{'date': 'April 2017',
 'description': 'Disentanglement test Sprites dataset.Procedurally generated 2D shapes, from 6 disentangled latent factors.This dataset uses 6 latents, controlling the color, shape, scale, rotation and position of a sprite. All possible variations of the latents are present. Ordering along dimension 1 is fixed and can be mapped back to the exact latent values that generated that image.We made sure that the pixel outputs are different. No noise added.',
 'version': 1,
 'latents_names': ('color', 'shape', 'scale', 'orientation', 'posX', 'posY'),
 'latents_possible_values': {'orientation': array([0.        , 0.16110732, 0.32221463, 0.48332195, 0.64442926,
         0.80553658, 0.96664389, 1.12775121, 1.28885852, 1.44996584,
         1.61107316, 1.77218047, 1.93328779, 2.0943951 , 2.25550242,
         2.41660973, 2.57771705, 2.73882436, 2.89993168, 3.061039  ,
         3.22214631, 3.38325363, 3.54436094, 3.70546826, 3.86657557,
         4.02768289, 4.1887902 , 4.34989